In [1]:
import pandas as pd
import numpy as np

import os
os.chdir('C:\\Users\\peter.casey\\Documents\\RodentAbatement')

In [2]:
data = pd.read_csv('rodent_model_data.csv')

In [3]:
data = data[data.year==2016]
data = data[data.calls==1]

Match Blocks to Wards

In [4]:
import geopandas as gpd

blocks = gpd.read_file('Census_Blocks__2010.geojson')[['GEOID', 'geometry']]
#blocks['TRACT'] = blocks['GEOID'].str[0:11]
tracts = gpd.read_file('Census_Tracts_in_2010.geojson')[['GEOID', 'TRACT', 'geometry']]
wards = gpd.read_file('Ward_from_2012.geojson')[['WARD', 'geometry']]

In [5]:
blocks_to_ward = gpd.sjoin(blocks, wards, how='left', op='intersects')
blocks_to_ward['WARD'] = blocks_to_ward['WARD'].astype(int)
print(blocks_to_ward.isnull().sum())
print(blocks_to_ward.WARD.value_counts().sort_index())
print(blocks_to_ward.GEOID.value_counts().head())

GEOID          0
geometry       0
index_right    0
WARD           0
dtype: int64
1     418
2    1009
3     919
4    1008
5     984
6    1111
7     869
8     726
Name: WARD, dtype: int64
110010079011000    3
110010076015000    3
110010048011000    3
110010046002004    3
110010049011003    3
Name: GEOID, dtype: int64


In [6]:
blocks_to_ward.head()

,GEOID,geometry,index_right,WARD
0,110010003003006,"POLYGON ((-77.07863030944546 38.9161507873491,...",6,3
0,110010003003006,"POLYGON ((-77.07863030944546 38.9161507873491,...",3,2
1,110010003003007,POLYGON ((-77.07698430927661 38.91551178808869...,3,2
2,110010003003008,POLYGON ((-77.07674330851466 38.91301678672006...,3,2
3,110010003003009,POLYGON ((-77.07412130781374 38.91268078711014...,3,2


In [7]:
best_ward = pd.DataFrame(blocks_to_ward.groupby('GEOID').WARD.max())
best_ward = best_ward.reset_index()
#best_ward['GEOID'] = best_ward.GEOID.astype(int)
best_ward.to_csv('best_ward.csv', index=False)
best_ward = pd.read_csv('best_ward.csv')
best_ward.GEOID.value_counts().head()

110010081003001    1
110010050022002    1
110010071001008    1
110010103001009    1
110010078083002    1
Name: GEOID, dtype: int64

In [8]:
final_data = data.merge(best_ward, how='left', on='GEOID')

In [9]:
for c in final_data.columns:
    if 'month_' in c:
        final_data = final_data.drop([c], axis=1)
    elif 'year' in c:
        final_data = final_data.drop([c], axis=1)
    else:
        continue

In [10]:
final_data = final_data[['activity', 'alley_condition', 'bbl_hotel', 'bbl_multifamily_rental',
       'bbl_restaurant', 'bbl_single_family_rental', 'bbl_storage',
       'bbl_two_family_rental', 'communitygarden_area', 'communitygarden_id', 
        'dcrapermit_addition', 'dcrapermit_demolition', 'dcrapermit_excavation', 
       'dcrapermit_new_building', 'dcrapermit_raze', 'impervious_area', 'month', 'num_mixed_use',
       'num_non_residential', 'num_residential', 'park', 'pct_mixed_use', 'pct_non_residential',
       'pct_residential', 'pop_density', 'sidewalk_grates', 'ssl_cndtn_Average_comm',
       'ssl_cndtn_Average_res', 'ssl_cndtn_Excellent_comm',
       'ssl_cndtn_Excellent_res', 'ssl_cndtn_Fair_comm', 'ssl_cndtn_Fair_res',
       'ssl_cndtn_Good_comm', 'ssl_cndtn_Good_res', 'ssl_cndtn_Poor_comm',
       'ssl_cndtn_Poor_res', 'ssl_cndtn_VeryGood_comm',
       'ssl_cndtn_VeryGood_res', 'tot_pop', 'well_activity', 'WARD']]

In [11]:
final_data.columns

Index(['activity', 'alley_condition', 'bbl_hotel', 'bbl_multifamily_rental',
       'bbl_restaurant', 'bbl_single_family_rental', 'bbl_storage',
       'bbl_two_family_rental', 'communitygarden_area', 'communitygarden_id',
       'dcrapermit_addition', 'dcrapermit_demolition', 'dcrapermit_excavation',
       'dcrapermit_new_building', 'dcrapermit_raze', 'impervious_area',
       'month', 'num_mixed_use', 'num_non_residential', 'num_residential',
       'park', 'pct_mixed_use', 'pct_non_residential', 'pct_residential',
       'pop_density', 'sidewalk_grates', 'ssl_cndtn_Average_comm',
       'ssl_cndtn_Average_res', 'ssl_cndtn_Excellent_comm',
       'ssl_cndtn_Excellent_res', 'ssl_cndtn_Fair_comm', 'ssl_cndtn_Fair_res',
       'ssl_cndtn_Good_comm', 'ssl_cndtn_Good_res', 'ssl_cndtn_Poor_comm',
       'ssl_cndtn_Poor_res', 'ssl_cndtn_VeryGood_comm',
       'ssl_cndtn_VeryGood_res', 'tot_pop', 'well_activity', 'WARD'],
      dtype='object')

In [12]:
final_data.to_csv('C:\\Users\\peter.casey\\Documents\\dspp\\rat_data_2016.csv', index=False)

## Messing around

In [82]:
blocks_to_ward = blocks_to_ward.reset_index()
blocks_to_ward['geometry'].intersection(wards['geometry'].iloc[int(blocks_to_ward.index_right)]).area

TypeError: cannot convert the series to <class 'int'>

In [ ]:
intersect_areas = gpd.overlay(tracts, wards, how = 'intersection').area

In [ ]:
tract_areas = tracts.merge(pd.DataFrame(intersect_areas, columns=['area']), how='left', left_index=True, right_index=True)
max_area = pd.DataFrame(tract_areas.groupby('GEOID').area.max(), columns=['area'])
tract_areas.merge(tracts, how='inner', left_on='area', right_on='area')

In [36]:
address_to_block = blocks.merge(mar, how='left', left_index=True, right_index=True)
address_to_block.head().T

GEOID,110010001001000,110010001001000,110010001001000,110010001001000,110010001001000
geometry,POLYGON ((-77.06474566621758 38.91916571557175...,POLYGON ((-77.06474566621758 38.91916571557175...,POLYGON ((-77.06474566621758 38.91916571557175...,POLYGON ((-77.06474566621758 38.91916571557175...,POLYGON ((-77.06474566621758 38.91916571557175...
ADDRESS_ID,261707,273153,273155,273157,279450
MARID,261707,273153,273155,273157,279450
STATUS,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE
ADDRNUM,3001,1601,2517,2521,1901
STNAME,R,28TH,Q,Q,WISCONSIN
STREET_TYPE,STREET,STREET,STREET,STREET,AVENUE
QUADRANT,NW,NW,NW,NW,NW
FULLADDRESS,3001 R STREET NW,1601 28TH STREET NW,2517 Q STREET NW,2521 Q STREET NW,1901 WISCONSIN AVENUE NW
CITY,WASHINGTON,WASHINGTON,WASHINGTON,WASHINGTON,WASHINGTON


In [38]:
address_to_block['ward'] = address_to_block.WARD.str[-1]
address_to_block.reset_index().groupby('GEOID').ward.max()

GEOID
110010001001000      2
110010001001001      2
110010001001002      2
110010001001003      2
110010001001004      2
110010001001005      2
110010001001006      2
110010001001007      2
110010001001008      2
110010001001009      2
110010001001010      2
110010001001011      2
110010001002000      2
110010001002001      2
110010001002002      2
110010001002003      2
110010001002004      2
110010001002005    NaN
110010001002006      2
110010001002007      2
110010001002008      2
110010001002009      2
110010001003000      2
110010001003001      2
110010001003002      2
110010001003003      2
110010001003004      2
110010001003005      2
110010001003006      2
110010001003007      2
                  ... 
110010111003028      5
110010111003029      5
110010111003030    NaN
110010111003031      5
110010111003032      5
110010111003033    NaN
110010111003034    NaN
110010111003035    NaN
110010111003036    NaN
110010111003037      5
110010111003038      5
110010111003039      5
11001

In [ ]:
def get_size_of_intersection(row, soil_type):
    return row['geometry'].intersection(soil_type['geometry'].iloc[int(row['index_right'])]).area

combined['intersection_size'] = combined.apply(lambda row : 
                                       get_size_of_intersection(row, soil_type), axis=1)